In [770]:
#import libraries used into
from datetime import datetime, timedelta
import pandas as pd, numpy as np, os, time
from pydrive2.auth import GoogleAuth
from pydrive2.drive import GoogleDrive
from pydrive2.files import FileNotUploadedError
directorio_credenciales = 'credentials_module.json'

In [771]:
def login():
    GoogleAuth.DEFAULT_SETTINGS['client_config_file'] = directorio_credenciales
    gauth = GoogleAuth()
    gauth.LoadCredentialsFile(directorio_credenciales)
    
    if gauth.credentials is None:
        gauth.LocalWebserverAuth(port_numbers=[8092])
    elif gauth.access_token_expired:
        gauth.Refresh()
    else:
        gauth.Authorize()
        
    gauth.SaveCredentialsFile(directorio_credenciales)
    credenciales = GoogleDrive(gauth)
    return credenciales
def subir_archivo(ruta_archivo,id_folder):
    credenciales = login()
    archivo = credenciales.CreateFile({'parents': [{"kind": "drive#fileLink",\
                                                    "id": id_folder}]})
    archivo['title'] = ruta_archivo.split("/")[-1]
    archivo.SetContentFile(ruta_archivo)
    archivo.Upload()
def bajar_archivo_por_id(id_drive,ruta_descarga):
    credenciales = login()
    archivo = credenciales.CreateFile({'id': id_drive}) 
    nombre_archivo = archivo['title']
    archivo.GetContentFile(ruta_descarga + nombre_archivo)
def busca(query):
    resultado = []
    credenciales = login()
    # Archivos con el nombre 'mooncode': title = 'mooncode'
    # Archivos que contengan 'mooncode' y 'mooncoders': title contains 'mooncode' and title contains 'mooncoders'
    # Archivos que NO contengan 'mooncode': not title contains 'mooncode'
    # Archivos que contengan 'mooncode' dentro del archivo: fullText contains 'mooncode'
    # Archivos en el basurero: trashed=true
    # Archivos que se llamen 'mooncode' y no esten en el basurero: title = 'mooncode' and trashed = false
    lista_archivos = credenciales.ListFile({'q': query}).GetList()
    for f in lista_archivos:
        # ID Drive
        print('ID Drive:',f['id'])
        # Link de visualizacion embebido
        print('Link de visualizacion embebido:',f['embedLink'])
        # Nombre del archivo
        print('Nombre del archivo:',f['title'])
        # Tipo de archivo
        print('Tipo de archivo:',f['mimeType'])
        # Esta en el basurero
        print('Esta en el basurero:',f['labels']['trashed'])
        # Fecha de creacion
        print('Fecha de creacion:',f['createdDate'])
        # Fecha de ultima modificacion
        print('Fecha de ultima modificacion:',f['modifiedDate'])
        # Version
        print('Version:',f['version'])
        # Tamanio
        print('Tamanio:',f['fileSize'])
        resultado.append(f)
    
    return resultado
def borrar_recuperar(id_archivo):
    credenciales = login()
    archivo = credenciales.CreateFile({'id': id_archivo})
    # MOVER A BASURERO
    #archivo.Trash()
    # SACAR DE BASURERO
    #archivo.UnTrash()
    # ELIMINAR PERMANENTEMENTE
    archivo.Delete()

In [772]:
OT = busca("fullText contains 'OT'");

ID Drive: 1GU62-GL3TTF8kUbt2h2aciqFXqh7RDlJ
Link de visualizacion embebido: https://drive.google.com/file/d/1GU62-GL3TTF8kUbt2h2aciqFXqh7RDlJ/preview?usp=drivesdk
Nombre del archivo: OT 25.11.2022.XLSX
Tipo de archivo: application/vnd.openxmlformats-officedocument.spreadsheetml.sheet
Esta en el basurero: False
Fecha de creacion: 2022-11-25T18:38:46.865Z
Fecha de ultima modificacion: 2022-11-25T15:25:00.000Z
Version: 5
Tamanio: 3279768
ID Drive: 1wxzd2pqHw8BD-lmtEibc51NNSsbxSNyN
Link de visualizacion embebido: https://drive.google.com/file/d/1wxzd2pqHw8BD-lmtEibc51NNSsbxSNyN/preview?usp=drivesdk
Nombre del archivo: OT 25.11.2022.XLSX
Tipo de archivo: application/vnd.openxmlformats-officedocument.spreadsheetml.sheet
Esta en el basurero: False
Fecha de creacion: 2022-11-25T17:10:55.544Z
Fecha de ultima modificacion: 2022-11-25T17:10:55.544Z
Version: 3
Tamanio: 3279768
ID Drive: 1WjLVCsHsRZnzx9yGuyfhkzx5l7qLJTZI
Link de visualizacion embebido: https://drive.google.com/file/d/1WjLVCsHsRZnzx

In [773]:
df = pd.DataFrame(OT)
df.drop(df[~df['title'].str.contains('OT')].index, inplace = True)


In [774]:
df = df.sort_values( by = ['createdDate'],ascending=False)
print("The sorted list is : \n{}".format(df['createdDate']))
print("The newest date is: {}".format(df['createdDate'].max()))
first_date = df['createdDate'].iloc[0]
first_id = df['id'].iloc[0]
second_date = df['createdDate'].iloc[1]
second_id = df['id'].iloc[1]
print("the first date is '{}' with an id '{}', and the second one is '{}' with an id '{}'".format(first_date,first_id,second_date,second_id))

The sorted list is : 
8     2022-11-25T21:08:17.335Z
2     2022-11-25T21:08:15.188Z
9     2022-11-25T19:15:53.997Z
4     2022-11-25T19:15:51.420Z
10    2022-11-25T19:00:51.400Z
3     2022-11-25T19:00:48.197Z
11    2022-11-25T18:38:49.895Z
0     2022-11-25T18:38:46.865Z
1     2022-11-25T17:10:55.544Z
12    2022-11-24T23:15:19.839Z
6     2022-11-24T23:15:16.847Z
13    2022-11-24T23:15:03.592Z
7     2022-11-24T23:15:00.159Z
Name: createdDate, dtype: object
The newest date is: 2022-11-25T21:08:17.335Z
the first date is '2022-11-25T21:08:17.335Z' with an id '1RbB4CuinpfWFHe_aJJEc2gdwjhyqeJFj', and the second one is '2022-11-25T21:08:15.188Z' with an id '1WjLVCsHsRZnzx9yGuyfhkzx5l7qLJTZI'


In [775]:
main_path = "/mnt/c/Users/mruiz/Documents/DB OT/"
main_path_ = os.listdir(main_path)
main_path_.remove("CurrentDocuments")
bajar_archivo_por_id(first_id,main_path)
pd_frame = {
    'File':[],
    'Date':[]
}

for element in main_path_:
    path = main_path + str(element)
    timestamp = os.path.getctime(path) #https://flaviocopes.com/python-get-file-details/
    #timestamp = time.ctime(timestamp)
    pd_frame['File'].append(element)
    pd_frame['Date'].append(timestamp)

In [776]:
data_db = pd.DataFrame(pd_frame)
#print(data_db['Date'].dtype)
data_db['Date'] = pd.to_datetime(data_db['Date'],unit='s')
data_db = data_db.sort_values(by = ['Date'],ascending=False)
#print(data_db['Date'])
actual_data = data_db['File'].iloc[0]
compare_data = data_db['File'].iloc[1]
print("Los datos actuales son '{}', y los datos a comparar son '{}'".format(actual_data,compare_data))

Los datos actuales son 'OT 18.11.2022 5 P.M. - Miguel Ruiz.XLSX', y los datos a comparar son 'OT 16.11.2022 - Miguel Ruiz.XLSX'


In [777]:
current_path = "/mnt/c/Users/mruiz/Documents/DB OT/CurrentDocuments"
current_month = datetime.now().strftime('%B')
current_path_ = os.listdir(current_path)
main_path_ = os.listdir(main_path)
data_yesterday_excel = {}
data_am_excel = {}
repro_nuevas = 1

In [778]:
def get_data(entry_data):
    file = main_path + entry_data
    filexcel = pd.read_excel(file,sheet_name="Sheet1")
    print("Los datos del archivo ", entry_data, " fueron procesados")
    return filexcel

In [779]:
while True:
    try:
        data_yesterday_excel = get_data(compare_data)
        data_am_excel = get_data(actual_data)
        break
    except FileNotFoundError:
        print("Hay al menos un archivo que no fue leído: ")
        print("Archivo Ayer : ", type(data_yesterday_excel))
        print("Archivo hoy por la mañana: ", type(data_am_excel))
        break

Los datos del archivo  OT 16.11.2022 - Miguel Ruiz.XLSX  fueron procesados
Los datos del archivo  OT 18.11.2022 5 P.M. - Miguel Ruiz.XLSX  fueron procesados


In [780]:
if type(data_am_excel) == type(data_yesterday_excel):
    programa = data_am_excel
    print("Los datos de la mañana fueron seleccionados")
else :
    print("Error")

Los datos de la mañana fueron seleccionados


In [781]:
data2 =current_path + "/Master_OT.xlsx"
nombres = pd.read_excel(data2, sheet_name="Names", header=0)

In [782]:
data3 = current_path + "/Politicas.XLSX"
politicas = pd.read_excel(data3, sheet_name="POLITICAS CONTRATO", header=0)

In [783]:
data7 = current_path + "/Ubicaciones_Equipos.xlsx"
ubicaciones = pd.read_excel(data7, sheet_name="Sheet1", header=0)

In [784]:
data8 = current_path + "/Reprogramaciones.xlsx"
reprogramaciones_ = pd.read_excel(data8, sheet_name="DB", header=0)

In [785]:
reprogramaciones = data_yesterday_excel.copy()

In [786]:
reprogramaciones.rename(columns={'Inicio programado':'Fecha_reprogramada'}, inplace=True)

In [787]:
programa['Material'] = np.where((programa['Clase de orden'] == 'ZMCP') &  (programa['Texto breve'].str.contains('Emer|Revi', case=False)), 'NO',
np.where(programa['Clase de orden'] == 'ZPM2','SI',
np.where(programa['Clase de orden'] == 'ZMCP','SI',
np.where(programa['Clase de orden'] == 'ZMCI', 'SI',
np.where((programa['Clase de orden'] == 'ZPM1') &  (programa['Texto breve'].str.contains('CAFA|CABAT|ACUM', case=False)), 'SI','NO')))))

In [788]:
#print('Las columnas de programa son', list(programa.columns.values))
#print('Las columnas de nombres son', list(nombres.columns.values))
#print('Las columnas de politicas son', list(politicas.columns.values))
#print('Las columnas de material local son', list(mlocal.columns.values))
#print('Las columnas de material foraneo son', list(mforaneo.columns.values))
#print('Las columnas de las repro son ',list(reprogramaciones.columns.values))
#print('Las columnas de reportes son ', list(reportes.columns.values))
#print('Las columnas de ubicaciones son', list(ubicaciones.columns.values))
#print(list(result_df.columns.values))

COMIENZAN LOS MERGE

In [789]:
repro_columns = ['Fecha_reprogramada', 'Orden']

In [790]:
result_df = pd.merge(programa,reprogramaciones[repro_columns],on='Orden',how='left' )

In [791]:
result_df = pd.merge(left=result_df,right=nombres,left_on='Pto.tbjo.responsable', right_on='TecID',how='left')

In [792]:
politicas_columns = ['Equipo','gestora archivo anahi ', 'reportes archivo anahi ', 'Tipo Equipo', 'Frecuencia', 
'Requiere Acceso', 'Tiempo aviso previo', 'Dia o fecha del Serv.', 'Horario', 'Curso', 'DC-3', 
'Examenes Covid, antidoping, etc', 'Tiempo valido de prueba', 'Tecnicos para servicio', 'Requerimientos adicionales', 
'Correo solicitud acceso', 'FACTURA FIRMADA', 'FOTOS', 'REPORTE ESPECIAL', 'Incluye materiales', 'OC', 'FRECUENCIA DE OC', 
'PREVENTIVOS', 'CORRECTIVOS']

In [793]:
result_df = pd.merge(result_df,politicas[politicas_columns],on='Equipo',how='left')

In [794]:
ubicaciones_columns = ['Equipo', 'Dirección', 'Latitud', 'Longitud']

In [795]:
result_df = pd.merge(result_df,ubicaciones[ubicaciones_columns],on='Equipo',how='left')

TERMINAN LOS MERGE

In [796]:
print(list(result_df.columns.values))

['Inicio programado', 'Orden', 'Plan mant.preventivo', 'Pto.tbjo.responsable', 'Status del sistema', 'Texto breve', 'Cliente', 'Denominación de objeto técnico', 'Número de serie', 'Equipo', 'Clase de orden', 'Fe.liberación real', 'Fecha inicio real', 'Material', 'Fecha_reprogramada', 'TecID', 'Short name', 'Estatus', 'City', 'Center', 'gestora archivo anahi ', 'reportes archivo anahi ', 'Tipo Equipo', 'Frecuencia', 'Requiere Acceso', 'Tiempo aviso previo', 'Dia o fecha del Serv.', 'Horario', 'Curso', 'DC-3', 'Examenes Covid, antidoping, etc', 'Tiempo valido de prueba', 'Tecnicos para servicio', 'Requerimientos adicionales', 'Correo solicitud acceso', 'FACTURA FIRMADA', 'FOTOS', 'REPORTE ESPECIAL', 'Incluye materiales', 'OC', 'FRECUENCIA DE OC', 'PREVENTIVOS', 'CORRECTIVOS', 'Dirección', 'Latitud', 'Longitud']


In [797]:
result_df.assign(reprogramacion=0);

In [798]:
reprogramaciones = result_df.copy()

In [799]:
result_df['reprogramacion'] = np.where(result_df['Inicio programado']==reprogramaciones['Fecha_reprogramada'], "Sin_Reprogramar","Reprogramada")
result_df['reprogramacion'] = np.where(result_df['Fecha_reprogramada'].isnull(),"Sin estatus",result_df['reprogramacion'])

In [800]:
print(result_df['reprogramacion'].unique())

['Sin_Reprogramar' 'Sin estatus' 'Reprogramada']


In [801]:
result_df['Politica'] = np.where(result_df['Requiere Acceso'].isnull(),"NO","SI")
result_df['Servicio'] = np.where(result_df['Orden'].isnull(), int("0"),int("1"))

In [802]:
print(result_df.shape[0])
print(programa.shape[0])

101185
101185


In [ ]:
reprogramaciones = result_df[result_df['reprogramacion'].isin(['Reprogramada'])]
if reprogramaciones_['Orden'].iloc[0] == reprogramaciones['Orden'].iloc[0]:
       repro_nuevas = 0
       print("Sin Reprogramaciones nuevas")
       reprogramaciones = reprogramaciones_
else:
       reprogramaciones = reprogramaciones.assign(Dia_reprogramado = datetime.today().strftime('%Y-%m-%d'))
       reprogramaciones = pd.concat([reprogramaciones,reprogramaciones_])
       print(reprogramaciones.shape[0])

In [ ]:
reprogramaciones = reprogramaciones[['Orden', 'Inicio programado','Plan mant.preventivo', 'Pto.tbjo.responsable',
       'Status del sistema', 'Texto breve', 'Cliente', 'Fecha inicio real',
       'Fecha_reprogramada','Dia_reprogramado']]

In [ ]:
rep_path = current_path + "/" + "Reprogramaciones.xlsx"
if repro_nuevas == 0 :
    print("Se conservó el documento")
elif repro_nuevas != 0 : 
    os.remove(current_path + "/" + "Reprogramaciones.xlsx")   
    reprogramaciones.to_excel(rep_path, index=False,header=True)
    with pd.ExcelWriter(rep_path) as writer:
        reprogramaciones.to_excel(writer,index=False,sheet_name="DB")
        subir_archivo(rep_path,'1apsMV4sccu7h3GUmKsuX-xSII1UFhfop')

In [ ]:
repro_list = busca("title = 'Reprogramaciones.xlsx'");
repro_frame = pd.DataFrame(repro_list);
#print(list(main_frame.columns))

In [ ]:
repro_frame = repro_frame.sort_values( by = ['createdDate'],ascending=False)
print("The sorted list is : \n{}".format(repro_frame['createdDate']))
print("The newest date is: {}".format(repro_frame['createdDate'].max()))
first_date = repro_frame['createdDate'].iloc[0]
first_id = repro_frame['id'].iloc[0]
print("the first date is '{}' with an id '{}'".format(first_date,first_id))
keys = list(repro_frame['labels'].iloc[0].keys())
values = list(repro_frame['labels'].iloc[0].values())
mydict = {
    'status':keys,
    'value':values
}
print(mydict)
if mydict['value'][2] == True:
    print("All files are already erased")
else:
    borrar_recuperar(first_id)  
    print("File '{}' are erased".format(first_id))

In [ ]:
if repro_nuevas == 0 :
    print("File just added is working")
else:
    subir_archivo(rep_path,'1apsMV4sccu7h3GUmKsuX-xSII1UFhfop')
    print("New file uploaded succesfully")

In [ ]:
print(list(result_df.columns.values))

In [ ]:
result_df = result_df[['Inicio programado', 'Orden', 'reprogramacion', 'Fecha_reprogramada',    'Plan mant.preventivo', 'Pto.tbjo.responsable',
 'Short name','Estatus', 'City', 'Status del sistema','Servicio','Fe.liberación real', 'Fecha inicio real', 'Texto breve',
'Cliente', 'Denominación de objeto técnico', 'Número de serie', 'Equipo', 'Dirección',
'Politica', 'gestora archivo anahi ', 'Frecuencia', 'Requiere Acceso', 'Tiempo aviso previo', 'Dia o fecha del Serv.',
'Horario', 'Curso', 'DC-3', 'Examenes Covid, antidoping, etc', 'Tiempo valido de prueba','Tecnicos para servicio', 'Requerimientos adicionales',
'Correo solicitud acceso', 'FACTURA FIRMADA', 'FOTOS', 'REPORTE ESPECIAL', 'Incluye materiales', 'OC','FRECUENCIA DE OC', 'PREVENTIVOS', 'CORRECTIVOS',
'Material', 'Clase de orden']]

In [ ]:
os.remove(current_path + "/" + "Main.xlsx")
main_path = current_path + "/" + "Main.xlsx"   
result_df.to_excel(main_path, index=False,header=True)
with pd.ExcelWriter(main_path) as writer:
    result_df.to_excel(writer,index=False,sheet_name="DB")
print("Main was created succesfully")

In [ ]:
main_list = busca("title = 'Main.xlsx'");
main_frame = pd.DataFrame(main_list);
#print(list(main_frame.columns))

In [ ]:
main_frame = main_frame.sort_values( by = ['createdDate'],ascending=False)
print("The sorted list is : \n{}".format(main_frame['createdDate']))
print("The newest date is: {}".format(main_frame['createdDate'].max()))
first_date = main_frame['createdDate'].iloc[0]
first_id = main_frame['id'].iloc[0]
print("the first date is '{}' with an id '{}'".format(first_date,first_id))
keys = list(main_frame['labels'].iloc[0].keys())
values = list(main_frame['labels'].iloc[0].values())
mydict = {
    'status':keys,
    'value':values
}
print(mydict)
if mydict['value'][2] == True:
    print("All files are already erased")
else:
    borrar_recuperar(first_id)  
    print("File '{}' are erased".format(first_id))

In [ ]:
subir_archivo(main_path,'1apsMV4sccu7h3GUmKsuX-xSII1UFhfop')
print("Upload succesfully")